In [1]:
import cv2
import numpy as np
import face_recognition

Step 1 from the previous notebook

In [2]:
img_bill_gates = face_recognition.load_image_file("Images Basic/bill_gates_1.jpg")
img_bill_gates = cv2.cvtColor(img_bill_gates, cv2.COLOR_BGR2RGB)

The problem over here is that everytime we want to add new faces, we would have to change the names manually and then find it's encoding which would be quite messy. **Solution**: We will create a list which will get all the images from the folder and will find encodings accordingly.

In [3]:
import os

OS module helps us to interact with the files

In [10]:
# Images folder
path = "Images"

# Array of images
images = []

# Names for images
imagesNames = []

myList = os.listdir(path)
print(myList)

['bill_gates.jpg', 'elon_musk.jpg', 'jeff_bezos.jpg']


### Step 1: Read All Images

In [11]:
# reading all images
for img in myList:
    new_image = cv2.imread(path+"/"+img)
    images.append(new_image)
    imagesNames.append(os.path.splitext(img)[0])
    
print(imagesNames)

['bill_gates', 'elon_musk', 'jeff_bezos']


### Step 2: Finding Encodings

In [12]:
# function to find the encodings for the images

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

knownEncodings = findEncodings(images)
print(len(knownEncodings))

3


### Step 3: Finding Matches

Since, here we don't have any picture to match with, we will initialize the webcam first to take the pictures and then will move on to the face matching part.

We will iterate through all the faces in a current frame and then will match with all the known encodings.

In [9]:
# initializing webcam

cap = cv2.VideoCapture(0)
 
while True:
    success, img = cap.read()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    # since we can have multiple faces
    facesCurrFrame = face_recognition.face_locations(imgS)
    encodeCurrFrame = face_recognition.face_encodings(imgS, facesCurrFrame)
    
    for encodeFace,faceLoc in zip(encodeCurrFrame,facesCurrFrame):
        matches = face_recognition.compare_faces(knownEncodings,encodeFace)
        faceDis = face_recognition.face_distance(knownEncodings,encodeFace)
#         print(faceDis)
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = imagesNames[matchIndex].upper()
#             print(name)
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            
    cv2.imshow('Webcam',img)
    k = cv2.waitKey(1)  & 0xFF

    # press q to quit
    if k == ord('q'):  
        break
        
cap.release()          
cv2.destroyAllWindows() 